## Seldon V2 Kubernetes Examples

 * Create a kubernetes cluster with local auth to it
 * Install Kafka - see `kafka/strimzi` folder
 * Build if needed and place `seldon` binary in your path
 * Install Seldon on Kubernetes
   * Run `make deploy-k8s` from top level folder


In [12]:
MESH_IP=!kubectl get svc seldon-mesh -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP=MESH_IP[0]
import os
os.environ['MESH_IP'] = MESH_IP
MESH_IP

'172.21.255.2'

### Model

In [13]:
!cat ./models/sklearn-iris-gs.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: iris
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/mlserver/iris"
  requirements:
  - sklearn


In [14]:
!kubectl create -f ./models/sklearn-iris-gs.yaml

model.mlops.seldon.io/iris created


In [15]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

model.mlops.seldon.io/iris condition met


In [16]:
!kubectl get model iris -n seldon-mesh -o jsonpath='{.status}' | jq -M .

{
  "conditions": [
    {
      "lastTransitionTime": "2022-04-18T13:59:52Z",
      "status": "True",
      "type": "ModelReady"
    },
    {
      "lastTransitionTime": "2022-04-18T13:59:52Z",
      "status": "True",
      "type": "Ready"
    }
  ]
}


In [17]:
!seldon model infer --model-name iris --inference-host ${MESH_IP} --inference-port 80 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

{
	"model_name": "iris_1",
	"model_version": "1",
	"id": "60268d20-582f-4772-81a8-d799c2b866ae",
	"parameters": null,
	"outputs": [
		{
			"name": "predict",
			"shape": [
				1
			],
			"datatype": "INT64",
			"parameters": null,
			"data": [
				2
			]
		}
	]
}


In [18]:
!seldon model infer --model-name iris --inference-mode grpc --inference-host ${MESH_IP} --inference-port 80 \
   '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' | jq -M .

{
  "modelName": "iris_1",
  "modelVersion": "1",
  "outputs": [
    {
      "name": "predict",
      "datatype": "INT64",
      "shape": [
        "1"
      ],
      "contents": {
        "int64Contents": [
          "2"
        ]
      }
    }
  ]
}


In [19]:
!kubectl get server mlserver -n seldon-mesh -o jsonpath='{.status}' | jq -M .

{
  "conditions": [
    {
      "lastTransitionTime": "2022-04-16T08:43:39Z",
      "status": "True",
      "type": "Ready"
    },
    {
      "lastTransitionTime": "2022-04-16T08:43:39Z",
      "reason": "StatefulSet replicas matches desired replicas",
      "status": "True",
      "type": "StatefulSetReady"
    }
  ],
  "loadedModels": 1
}


In [20]:
!kubectl delete -f ./models/sklearn-iris-gs.yaml

model.mlops.seldon.io "iris" deleted


### Experiment

In [21]:
!cat ./experiments/sklearn1.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: iris
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/mlserver/iris"
  requirements:
  - sklearn


In [22]:
!cat ./experiments/sklearn2.yaml 

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: iris2
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/mlserver/iris"
  requirements:
  - sklearn


In [23]:
!kubectl create -f ./experiments/sklearn1.yaml
!kubectl create -f ./experiments/sklearn2.yaml

model.mlops.seldon.io/iris created
model.mlops.seldon.io/iris2 created


In [24]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

model.mlops.seldon.io/iris condition met
model.mlops.seldon.io/iris2 condition met


In [25]:
!cat ./experiments/ab-default-model.yaml 

apiVersion: mlops.seldon.io/v1alpha1
kind: Experiment
metadata:
  name: experiment-sample
  namespace: seldon-mesh
spec:
  defaultModel: iris
  candidates:
  - modelName: iris
    weight: 50
  - modelName: iris2
    weight: 50


In [26]:
!kubectl create -f ./experiments/ab-default-model.yaml 

experiment.mlops.seldon.io/experiment-sample created


In [27]:
!kubectl wait --for condition=ready --timeout=300s experiment --all -n seldon-mesh

experiment.mlops.seldon.io/experiment-sample condition met


In [28]:
!seldon model infer --inference-host ${MESH_IP} --inference-port 80 -i 50 --model-name iris \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[iris2_1:20 iris_1:30]


In [29]:
!kubectl delete -f ./experiments/ab-default-model.yaml 
!kubectl delete -f ./experiments/sklearn1.yaml
!kubectl delete -f ./experiments/sklearn2.yaml

experiment.mlops.seldon.io "experiment-sample" deleted
model.mlops.seldon.io "iris" deleted
model.mlops.seldon.io "iris2" deleted


### Pipeline - model chain

In [54]:
!cat ./models/tfsimple1.yaml
!cat ./models/tfsimple2.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple1
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple2
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow


In [55]:
!kubectl create -f ./models/tfsimple1.yaml
!kubectl create -f ./models/tfsimple2.yaml

model.mlops.seldon.io/tfsimple1 created
model.mlops.seldon.io/tfsimple2 created


In [56]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

model.mlops.seldon.io/tfsimple1 condition met
model.mlops.seldon.io/tfsimple2 condition met


In [57]:
!cat ./pipelines/tfsimples.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: tfsimples
  namespace: seldon-mesh
spec:
  steps:
    - name: tfsimple1
    - name: tfsimple2
      inputs:
      - tfsimple1
      tensorMap:
        tfsimple1.outputs.OUTPUT0: INPUT0
        tfsimple1.outputs.OUTPUT1: INPUT1
  output:
    steps:
    - tfsimple2


In [58]:
!kubectl create -f ./pipelines/tfsimples.yaml

pipeline.mlops.seldon.io/tfsimples created


In [59]:
!kubectl wait --for condition=ready --timeout=300s pipeline --all -n seldon-mesh

pipeline.mlops.seldon.io/tfsimples condition met


In [60]:
!seldon pipeline infer -p tfsimples --inference-mode grpc --inference-host ${MESH_IP} --inference-port 80 \
    '{"model_name":"simple","inputs":[{"name":"INPUT0","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]}]}' | jq -M .

{
  "modelName": "tfsimple2_1",
  "modelVersion": "1",
  "outputs": [
    {
      "name": "OUTPUT0",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          2,
          4,
          6,
          8,
          10,
          12,
          14,
          16,
          18,
          20,
          22,
          24,
          26,
          28,
          30,
          32
        ]
      }
    },
    {
      "name": "OUTPUT1",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          2,
          4,
          6,
          8,
          10,
          12,
          14,
          16,
          18,
          20,
          22,
          24,
          26,
          28,
          30,
          32
        ]
      }
    }
  ],
  "rawOutputContents": [
    "AgAAAAQAAAAGAAAACAAAAAoAAAAMAAAADgAAABAAAAASAAAAFAAAABYAAAAYAAAAGgAAABwAAAAeAAAAIAAAAA==",
    "Ag

In [61]:
!kubectl delete -f ./pipelines/tfsimples.yaml

pipeline.mlops.seldon.io "tfsimples" deleted


In [62]:
!kubectl delete -f ./models/tfsimple1.yaml
!kubectl delete -f ./models/tfsimple2.yaml

model.mlops.seldon.io "tfsimple1" deleted
model.mlops.seldon.io "tfsimple2" deleted


### Pipeline - model join

In [72]:
!cat ./models/tfsimple1.yaml
!cat ./models/tfsimple2.yaml
!cat ./models/tfsimple3.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple1
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple2
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple3
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow


In [73]:
!kubectl create -f ./models/tfsimple1.yaml
!kubectl create -f ./models/tfsimple2.yaml
!kubectl create -f ./models/tfsimple3.yaml

model.mlops.seldon.io/tfsimple1 created
model.mlops.seldon.io/tfsimple2 created
model.mlops.seldon.io/tfsimple3 created


In [74]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

model.mlops.seldon.io/tfsimple1 condition met
model.mlops.seldon.io/tfsimple2 condition met
model.mlops.seldon.io/tfsimple3 condition met


In [75]:
!cat ./pipelines/tfsimples-join.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: join
  namespace: seldon-mesh
spec:
  steps:
    - name: tfsimple1
    - name: tfsimple2
    - name: tfsimple3      
      inputs:
      - tfsimple1.outputs.OUTPUT0
      - tfsimple2.outputs.OUTPUT1
      tensorMap:
        tfsimple1.outputs.OUTPUT0: INPUT0
        tfsimple2.outputs.OUTPUT1: INPUT1
  output:
    steps:
    - tfsimple3


In [76]:
!kubectl create -f ./pipelines/tfsimples-join.yaml

pipeline.mlops.seldon.io/join created


In [77]:
!kubectl wait --for condition=ready --timeout=300s pipeline --all -n seldon-mesh

pipeline.mlops.seldon.io/join condition met


In [78]:
!seldon pipeline infer -p join --inference-mode grpc --inference-host ${MESH_IP} --inference-port 80 \
    '{"model_name":"simple","inputs":[{"name":"INPUT0","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]}]}' | jq -M .

{
  "modelName": "tfsimple3_1",
  "modelVersion": "1",
  "outputs": [
    {
      "name": "OUTPUT0",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          2,
          4,
          6,
          8,
          10,
          12,
          14,
          16,
          18,
          20,
          22,
          24,
          26,
          28,
          30,
          32
        ]
      }
    },
    {
      "name": "OUTPUT1",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          2,
          4,
          6,
          8,
          10,
          12,
          14,
          16,
          18,
          20,
          22,
          24,
          26,
          28,
          30,
          32
        ]
      }
    }
  ],
  "rawOutputContents": [
    "AgAAAAQAAAAGAAAACAAAAAoAAAAMAAAADgAAABAAAAASAAAAFAAAABYAAAAYAAAAGgAAABwAAAAeAAAAIAAAAA==",
    "Ag

In [79]:
!kubectl delete -f ./pipelines/tfsimples-join.yaml

pipeline.mlops.seldon.io "join" deleted


In [80]:
!kubectl delete -f ./models/tfsimple1.yaml
!kubectl delete -f ./models/tfsimple2.yaml
!kubectl delete -f ./models/tfsimple3.yaml

model.mlops.seldon.io "tfsimple1" deleted
model.mlops.seldon.io "tfsimple2" deleted
model.mlops.seldon.io "tfsimple3" deleted
